In [ ]:
!pip install onnxruntime-genai
!pip install onnxruntime
!pip install olive
!pip install huggingface-hub

In [ ]:
!huggingface-cli login --token <TOKEN>

In [20]:
import olive.workflows

olive.workflows.run("olive.json")

[2024-04-10 18:59:28,424] [INFO] [run.py:243:run] Loading Olive module configuration from: C:\Users\nakersha\AppData\Local\miniconda3\envs\assistant\Lib\site-packages\olive\olive_config.json
[2024-04-10 18:59:28,430] [INFO] [run.py:249:run] Loading run configuration from: olive.json
[2024-04-10 18:59:28,438] [INFO] [config.py:184:validate_evaluate_input_model] No evaluator is specified, skip to evaluate model
[2024-04-10 18:59:28,443] [DEBUG] [accelerator.py:245:normalize_accelerators] The accelerator device and execution providers are specified, skipping deduce.
[2024-04-10 18:59:28,445] [DEBUG] [accelerator.py:275:normalize_accelerators] Supported execution providers for device cpu: ['CPUExecutionProvider']
[2024-04-10 18:59:28,448] [DEBUG] [accelerator.py:302:create_accelerators] Initial accelerators and execution providers: {'CPU': ['CPUExecutionProvider']}
[2024-04-10 18:59:28,451] [INFO] [accelerator.py:324:create_accelerators] Running workflow on accelerator specs: cpu-cpu
[2024

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Reading embedding layer
Reading decoder layer 0
Reading decoder layer 1
Reading decoder layer 2
Reading decoder layer 3
Reading decoder layer 4
Reading decoder layer 5
Reading decoder layer 6
Reading decoder layer 7
Reading decoder layer 8
Reading decoder layer 9
Reading decoder layer 10
Reading decoder layer 11
Reading decoder layer 12
Reading decoder layer 13
Reading decoder layer 14
Reading decoder layer 15
Reading decoder layer 16
Reading decoder layer 17
Reading decoder layer 18
Reading decoder layer 19
Reading decoder layer 20
Reading decoder layer 21
Reading decoder layer 22
Reading decoder layer 23
Reading decoder layer 24
Reading decoder layer 25
Reading decoder layer 26
Reading decoder layer 27
Reading decoder layer 28
Reading decoder layer 29
Reading decoder layer 30
Reading decoder layer 31
Reading final norm
Reading LM head
Saving ONNX model in C:\Users\nakersha\Develop\code\natke\assistant\cache\models\0_GenAIModelExporter-1473a6e460df1ddcd4cf088ff0019b1e-fe48ab55cdf4d03b

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Saving processing files in C:\Users\nakersha\Develop\code\natke\assistant\cache\models\0_GenAIModelExporter-1473a6e460df1ddcd4cf088ff0019b1e-fe48ab55cdf4d03b843ede7c3c3be27b-cpu-cpu\output_model for GenAI
[2024-04-10 19:05:32,649] [DEBUG] [resource_path.py:156:create_resource_path] Resource path C:\Users\nakersha\Develop\code\natke\assistant\cache\models\0_GenAIModelExporter-1473a6e460df1ddcd4cf088ff0019b1e-fe48ab55cdf4d03b843ede7c3c3be27b-cpu-cpu\output_model is inferred to be of type folder.
[2024-04-10 19:05:32,672] [INFO] [engine.py:951:_run_pass] Pass genai_exporter:GenAIModelExporter finished in 364.187977 seconds
[2024-04-10 19:05:32,679] [DEBUG] [engine.py:708:_cache_model] Cached model 0_GenAIModelExporter-1473a6e460df1ddcd4cf088ff0019b1e-fe48ab55cdf4d03b843ede7c3c3be27b-cpu-cpu to cache\models\0_GenAIModelExporter-1473a6e460df1ddcd4cf088ff0019b1e-fe48ab55cdf4d03b843ede7c3c3be27b-cpu-cpu.json
[2024-04-10 19:05:32,685] [DEBUG] [engine.py:791:_cache_run] Cached run for 1473a6e46

{AcceleratorSpec(accelerator_type=<Device.CPU: 'cpu'>, execution_provider='CPUExecutionProvider', vender=None, version=None, memory=None, num_cores=None): <olive.engine.footprint.Footprint at 0x24294f50050>}

In [11]:
import onnxruntime_genai as og
import time

In [14]:
print("Loading model...")
app_started_timestamp = time.time()

model = og.Model(f'example-models\microsoft\phi2-cpu-int4\genai_exporter\cpu-cpu_model')
model_loaded_timestamp  = time.time()

print("Model loaded in {:.2f} seconds".format(model_loaded_timestamp - app_started_timestamp))



Loading model...
Model loaded in 36.07 seconds


In [15]:
print("Loading tokenizer...")
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

print("Tokenizer created")

system_prompt = "You are a helpful assistant. Answer in one sentence."
text = "What is Dilithium?"

input_tokens = tokenizer.encode(system_prompt + text)

prompt_length = len(input_tokens)

Loading tokenizer...
Tokenizer created


In [16]:
started_timestamp = time.time()

print("Creating generator ...")
params = og.GeneratorParams(model)
params.set_search_options({"do_sample": False, "max_length": 2028, "min_length": 0, "top_p": 0.9, "top_k": 40, "temperature": 1.0, "repetition_penalty": 1.0})
params.input_ids = input_tokens
generator = og.Generator(model, params)
print("Generator created")

first = True
new_tokens = []

while not generator.is_done():
    generator.compute_logits()
    generator.generate_next_token()
    if first:
        first_token_timestamp = time.time()
        first = False

    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end="")
    new_tokens.append(new_token)

print()
run_time = time.time() - started_timestamp
print(f"Prompt tokens: {len(input_tokens)}, New tokens: {len(new_tokens)}, Time to first: {(first_token_timestamp - started_timestamp):.2f}s, New tokens per second: {len(new_tokens)/run_time:.2f} tps")


Creating generator ...
Generator created

A: Dilithium is a fictional substance in the Star Trek universe that is used as a propellant and a power source for spaceships.

Prompt tokens: 17, New tokens: 32, Time to first: 1.09s, New tokens per second: 3.83 tps


## Download and build llama.cpp
* git clone https://github.com/ggerganov/llama.cpp.git
* cd llama.cpp
* cmake -S . -B build/ -D CMAKE_BUILD_TYPE=Release
* cmake --build build/ --config Release

## Download gguf phi-2 model
* cd ..
* git clone https://huggingface.co/TheBloke/phi-2-GGUF

In [9]:
# Compare with llama.cpp.

! ..\..\ggerganov\llama.cpp\build\bin\Release\main -m ..\..\the\phi-2-GGUF\phi-2.Q4_K_M.gguf --prompt "You are a helpful assistant. Answer in one sentence. What is Dilithium?"


You are a helpful assistant. Answer in one sentence. What is Dilithium?
A: Dilithium is a powerful explosive used in the Star Trek universe.


Log start
main: build = 2647 (8228b66d)
main: built with MSVC 19.39.33523.0 for x64
main: seed  = 1712797839
llama_model_loader: loaded meta data with 20 key-value pairs and 325 tensors from ..\..\thebloke\phi-2-GGUF\phi-2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi2
llama_model_loader: - kv   1:                               general.name str              = Phi2
llama_model_loader: - kv   2:                        phi2.context_length u32              = 2048
llama_model_loader: - kv   3:                      phi2.embedding_length u32              = 2560
llama_model_loader: - kv   4:                   phi2.feed_forward_length u32              = 10240
llama_model_loader: - kv   5:                           phi2.block_count u32              = 32
llama_model_loader: - kv   6:                  phi2.attenti